In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Arrhythmia"
cohort = "GSE55231"

# Input paths
in_trait_dir = "../../input/GEO/Arrhythmia"
in_cohort_dir = "../../input/GEO/Arrhythmia/GSE55231"

# Output paths
out_data_file = "../../output/preprocess/Arrhythmia/GSE55231.csv"
out_gene_data_file = "../../output/preprocess/Arrhythmia/gene_data/GSE55231.csv"
out_clinical_data_file = "../../output/preprocess/Arrhythmia/clinical_data/GSE55231.csv"
json_path = "../../output/preprocess/Arrhythmia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide identification of expression quantitative trait loci (eQTLs) in human heart: gene expression"
!Series_summary	"In recent years genome-wide association studies (GWAS) have uncovered numerous chromosomal loci associated with various electrocardiographic traits and cardiac arrhythmia predisposition. A considerable fraction of these loci lie within inter-genic regions. Trait-associated SNPs located in putative regulatory regions likely exert their effect by modulating gene expression. Hence, the key to unraveling the molecular mechanisms underlying cardiac traits is to interrogate variants for association with differential transcript abundance by expression quantitative trait locus (eQTL) analysis. In this study we conducted an eQTL analysis of human heart. To this end, left ventricular mycardium samples from non-diseased human donor hearts were hybridized to Illumina HumanOmniExpress BeadChips for genotyping (n = 129) and Illumina Human H

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any, List
import numpy as np

# 1. Gene Expression Data Availability
# Based on background information, this is a human heart gene expression study using Illumina HT12 BeadChips
is_gene_available = True  # This dataset contains gene expression data

# 2.1 Data Availability
# Trait (Arrhythmia)
# There is no explicit arrhythmia data in the sample characteristics
# This is a normal heart study with no disease information in the metadata
trait_row = None  # Arrhythmia data is not available

# Age
age_row = 2  # Age information is available at index 2

# Gender
gender_row = 0  # Gender information is available at index 0

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> Optional[int]:
    """
    Convert arrhythmia status to binary values.
    0 = no arrhythmia, 1 = arrhythmia
    Since we don't have trait data, this function is defined but won't be used.
    """
    if value is None:
        return None
    
    # Extract value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Define conversion logic
    if value in ['yes', 'arrhythmia', 'present', 'positive', 'true', '1']:
        return 1
    elif value in ['no', 'none', 'absent', 'negative', 'false', '0']:
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """
    Convert age to continuous values (float).
    """
    if value is None:
        return None
    
    # Extract value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender to binary values.
    0 = female, 1 = male
    """
    if value is None:
        return None
    
    # Extract value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Define conversion logic
    if value in ['male', 'm', '1']:
        return 1
    elif value in ['female', 'f', '0']:
        return 0
    else:
        return None

# 3. Save Metadata
# Trait data is not available, so is_trait_available is False
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=(trait_row is not None)
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this step as instructed


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Arrhythmia/GSE55231/GSE55231_series_matrix.txt.gz


Gene data shape: (47323, 129)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers starting with "ILMN_" are Illumina probe IDs, not human gene symbols
# ILMN_ is the prefix used by Illumina microarray platforms
# These probe IDs need to be mapped to human gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '

Column 'ID' contains values matching gene data ID pattern


Column 'Species' contains values matching gene data ID pattern


Column 'Source' contains values matching gene data ID pattern


Column 'Transcript' contains values matching gene data ID pattern


Column 'Probe_Id' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Transcript' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'ILMN_Gene' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'Unigene_ID' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'Entrez_Gene_ID' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'Symbol' may contain gene-related information
Sample values: ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for probe IDs and gene symbols
prob_col = 'ID'  # Contains ILMN probe IDs matching gene expression data
gene_col = 'Symbol'  # Contains gene symbols for mapping

print(f"Using {prob_col} for probe IDs and {gene_col} for gene symbols")

# 2. Get a gene mapping dataframe by extracting the two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping shape: {gene_mapping.shape}")
print("Gene mapping sample:")
print(gene_mapping.head())

# Check for unmappable probes
unique_probes_in_gene_data = len(gene_data.index.unique())
unique_probes_in_mapping = len(gene_mapping['ID'].unique())
print(f"Unique probes in gene data: {unique_probes_in_gene_data}")
print(f"Unique probes in mapping: {unique_probes_in_mapping}")
print(f"Percentage of probes with mapping: {(len(set(gene_data.index) & set(gene_mapping['ID'])) / unique_probes_in_gene_data) * 100:.2f}%")

# 3. Convert probe-level measurements to gene expression data
gene_expression_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape (after mapping): {gene_expression_data.shape}")
print("First few gene symbols after mapping:")
print(gene_expression_data.index[:10])

# Update gene_data with the mapped gene expression data
gene_data = gene_expression_data

# Preview the gene data
print("\nGene data preview:")
print(preview_df(gene_data, n=3))

# Save the gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")


Using ID for probe IDs and Symbol for gene symbols


Gene mapping shape: (44837, 2)
Gene mapping sample:


             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Unique probes in gene data: 47323
Unique probes in mapping: 44837
Percentage of probes with mapping: 93.09%


Gene expression data shape (after mapping): (21464, 129)
First few gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')

Gene data preview:
{'GSM1332057': [10.127264694, 14.379807187, 15.035030803000001], 'GSM1332058': [9.944254334, 14.493606312, 14.046573366], 'GSM1332059': [10.142984839, 14.650354162, 14.734314932], 'GSM1332060': [10.091085048, 15.670301627, 14.974311387], 'GSM1332061': [10.265431286999998, 14.149712113, 15.11576243], 'GSM1332062': [10.542400339, 14.603533795, 14.706047456], 'GSM1332063': [9.877080485, 15.060291367000001, 14.514800183999998], 'GSM1332064': [9.966614693, 15.20830677, 14.662965185000001], 'GSM1332065': [9.924015967999999, 14.811890086, 13.821568969000001], 'GSM1332066': [9.13442078, 14.365565483000001, 14.005469864], 'GSM1332067': [9.995247309, 14.518232182, 14.380888581], 'GSM1332068': [10.199034656, 14.724974837, 15.108685398], 'GSM133206

Gene data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE55231.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to liver fibrosis progression, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for liver cirrhosis analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (21464, 129)
Gene data shape after normalization: (20259, 129)


Normalized gene expression data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE55231.csv


Original clinical data preview:
         !Sample_geo_accession                           GSM1332057  \
0  !Sample_characteristics_ch1                         gender: male   
1  !Sample_characteristics_ch1  tissue: left ventricular myocardium   
2  !Sample_characteristics_ch1                              age: 31   
3  !Sample_characteristics_ch1                            center: 3   

                            GSM1332058                           GSM1332059  \
0                       gender: female                         gender: male   
1  tissue: left ventricular myocardium  tissue: left ventricular myocardium   
2                              age: 54                              age: 32   
3                            center: 1                            center: 3   

                            GSM1332060                           GSM1332061  \
0                         gender: male                         gender: male   
1  tissue: left ventricular myocardium  tissue: left ventr